In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    NewsURLLoader,
    TrelloLoader,
)
from unstructured.cleaners.core import clean_extra_whitespace
from langchain.text_splitter import TokenTextSplitter
from dotenv import load_dotenv
load_dotenv()
from src.Microsofttodo import *
from src.summarize_text import SeperateTaskPrompt

Selected device: cuda


In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
import ast

from typing import Literal
from transformers import pipeline
from bs4 import BeautifulSoup
from src.Utils.utils import check_path, take_device

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    NewsURLLoader,
)
from unstructured.cleaners.core import clean_extra_whitespace
from langchain.text_splitter import TokenTextSplitter

from langchain_community.llms import CTransformers
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from ctransformers import AutoModelForCausalLM, AutoTokenizer
from langchain_openai import OpenAI
from datetime import datetime

class SeperateTaskPrompt:
    def __init__(self, template_path:str = 'config/seperate_task_template.txt', ):

        self.template_path = template_path
        with open(self.template_path, 'r') as file:
            self.template = file.read()
        self.load_llm()
        # self.prompt = PromptTemplate(template=self.template, input_variables=['date', 'question'])
        # self.llm_chain = LLMChain(prompt=self.prompt, llm=self.llm)

    def load_llm(self):
        self.llm = OpenAI(openai_api_key=OPENAI_API_KEY, 
                          max_tokens = 512,
                          temperature=0.7,
                          
                          )
        
    def get_response(self, text) -> list:

        tmp_prompt = f"{self.template}\n Now at {datetime.now()}, please break down this text: {text}"
        response = self.llm(tmp_prompt)
        # print('raw', response)
        response = self.get_tasks_from_string(response)
        return response

    def get_tasks_from_string(self, text:str) -> list:
        # Find the substring between '[' and ']'
        start_index = text.find('[')
        end_index = text.find(']') + 1
        json_string = text[start_index:end_index]
        tasks_list = eval(json_string)
        return tasks_list

In [5]:
seperator = SeperateTaskPrompt(template_path='config/todo_task_create_template.txt')


In [6]:
text = 'Hi remind me take my laptop in Hoa house after lunch, I have to call friends, ah no my parents at 10 o clock tomorrow. If done, go to the store at 4PM. Uhm I want to learn math tomorrow'

In [7]:
task = seperator.get_response(text)

/root/code_Bao/stock_price_4_fun/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [8]:
task

[{'title': 'Take laptop to Hoa house after lunch',
  'important': False,
  'dueDateTime': None},
 {'title': 'Call friends', 'important': False, 'dueDateTime': None},
 {'title': 'Call parents at 10 o clock tomorrow',
  'important': False,
  'dueDateTime': '2024-01-23:10:00:00'},
 {'title': 'Go to the store at 4 PM',
  'important': True,
  'dueDateTime': '2024-01-23:16:00:00'},
 {'title': 'Learn math tomorrow',
  'important': True,
  'dueDateTime': '2024-01-23:08:00:00'}]

In [9]:
print(task)

[{'title': "Take laptop to Hoa's house after lunch", 'important': False, 'dueDateTime': None}, {'title': 'Call friends', 'important': False, 'dueDateTime': None}, {'title': "Meet with parents at 10 o'clock tomorrow", 'important': False, 'dueDateTime': '2024-01-19:10:00:00'}, {'title': 'Go to the store at 4 PM', 'important': False, 'dueDateTime': '2024-01-18:16:00:00'}]


In [9]:
TRELLO_API_KEY= os.getenv('TRELLO_API_KEY')
TRELLO_TOKEN = os.getenv('TRELLO_TOKEN')

In [ ]:

# # Get the open cards from "Awesome Board"
# loader = TrelloLoader.from_credentials(
#     "SOW data",
#     api_key=TRELLO_API_KEY,
#     token=TRELLO_TOKEN,
#     card_filter="open",
# )
# documents = loader.load()

# print(documents[0].page_content)
# print(documents[0].metadata)

In [ ]:
# # Get all the cards from "Awesome Board" but only include the
# # card list(column) as extra metadata.
# loader = TrelloLoader.from_credentials(
#     "SOW data",
#     api_key=TRELLO_API_KEY,
#     token=TRELLO_TOKEN,
#     extra_metadata=("list"),
# )
# documents = loader.load()

# print(documents[0].page_content)
# print(documents[0].metadata)

In [14]:

class MicrosoftToDo:
    def __init__(self) -> None:
        pass
        
    def parse_response(self,response):
        return json.loads(response.content.decode())["value"]


    def get_lists(self):
        session = get_oauth_session()
        response = session.get(BASE_URL)
        response_value = self.parse_response(response)
        list = [item['displayName'] for item in response_value]
        print('Lists: ', list)
        return response_value



    def create_list(self, title: str):
        request_body = {"displayName": title}
        session = get_oauth_session()
        response = session.post(BASE_URL, json=request_body)
        return True if response.ok else response.raise_for_status()


    # TODO No associated command
    def rename_list(self, old_title: str, new_title: str):
        list_id = self.get_list_id_by_name(old_title)
        request_body = {"title": new_title}
        session = get_oauth_session()
        response = session.patch(f"{BASE_URL}/{list_id}", json=request_body)
        return True if response.ok else response.raise_for_status()


    def get_tasks(self, list_name: str = None, list_id: str = None, num_tasks: int = 100):
        assert (list_name is not None) or (
            list_id is not None
        ), "You must provide list_name or list_id"

        # For compatibility with cli
        if list_id is None:
            list_id = self.get_list_id_by_name(list_name)

        endpoint = (
            f"{BASE_URL}/{list_id}/tasks?$filter=status ne 'completed'&$top={num_tasks}"
        )
        session = get_oauth_session()
        response = session.get(endpoint)
        response_value = self.parse_response(response)
        return response_value

    def get_short_form_tasks(self, list_name: str = None, list_id: str = None, num_tasks: int = 100):
        new_response = []
        for task in self.get_tasks(list_name, list_id, num_tasks):
            new_task = {
                'importance': task.get('importance', None),
                'isReminderOn': task.get('isReminderOn', None),
                'title': task.get('title', None),
                'createdDateTime': task.get('createdDateTime', None),
                'hasAttachments': task.get('hasAttachments', None),
                'checklistItems': task.get('checklistItems', None),
                'body': task.get('body', None),
            }
            new_response.append(new_task)
        return new_response


    def complete_task(self,
                    list_name: str = None,
                    task_name: Union[str, int] = None,
                    list_id: str = None,
                    task_id: str = None,
                ):
        assert (list_name is not None) or (
            list_id is not None
        ), "You must provide list_name or list_id"
        assert (task_name is not None) or (
            task_id is not None
        ), "You must provide task_name or task_id"

        # For compatibility with cli
        if list_id is None:
            list_id = self.get_list_id_by_name(list_name)
        if task_id is None:
            task_id = self.get_task_id_by_name(list_name, task_name)

        endpoint = f"{BASE_URL}/{list_id}/tasks/{task_id}"
        request_body = {
            "status": TaskStatus.COMPLETED,
            "completedDateTime": datetime_to_api_timestamp(datetime.now()),
        }
        session = get_oauth_session()
        response = session.patch(endpoint, json=request_body)
        return True if response.ok else response.raise_for_status()


    def remove_task(self, list_name: str, task_name: Union[str, int]):
        list_id = self.get_list_id_by_name(list_name)
        task_id = self.get_task_id_by_name(list_name, task_name)
        endpoint = f"{BASE_URL}/{list_id}/tasks/{task_id}"
        session = get_oauth_session()
        response = session.delete(endpoint)
        return True if response.ok else response.raise_for_status()


    def get_list_id_by_name(self, list_name):
        endpoint = f"{BASE_URL}?$filter=startswith(displayName,'{list_name}')"
        session = get_oauth_session()
        response = session.get(endpoint)
        response_value = self.parse_response(response)
        try:
            return response_value[0]["id"]
        except IndexError:
            raise ListNotFound(list_name)


    def get_task_id_by_name(self, list_name: str, task_name: str):
        if isinstance(task_name, str):
            try:
                list_id = self.get_list_id_by_name(list_name)
                endpoint = f"{BASE_URL}/{list_id}/tasks?$filter=title eq '{task_name}'"
                session = get_oauth_session()
                response = session.get(endpoint)
                response_value = self.parse_response(response)
                return [Task(x) for x in response_value][0].id
            except IndexError:
                raise TaskNotFoundByName(task_name, list_name)
        # elif isinstance(task_name, int):
        #    tasks = get_tasks(list_name, task_list_position + 1)
        #    try:
        #        return tasks[task_list_position].id
        #    except IndexError:
        #        raise TaskNotFoundByIndex(task_list_position, list_name)
        else:
            raise

    def create_task(self,
                    task_name: str,
                    list_name: str = None,
                    list_id: str = None,
                    importance:bool = False,
                    dueDateTime: datetime = None,
                    body=None,
                    reminder_datetime: datetime = None,
                    ):
        assert (list_name is not None) or (
            list_id is not None
        ), "You must provide list_name or list_id"

        # For compatibility with cli
        if list_id is None:
            list_id = self.get_list_id_by_name(list_name)
        
        if dueDateTime is not None:
            dueDateTime = datetime.strptime(dueDateTime, '%Y-%m-%d:%H:%M:%S')
                # Check if the time part is '00:00'
            if dueDateTime.strftime('%H:%M') == '00:00':
                # Replace it with the current datetime
                dueDateTime = datetime.now()
                
            if reminder_datetime is None:
                reminder_datetime = dueDateTime
        
        importance = 'high' if importance else 'normal'

        if body is None:
            body ={"content": task_name,
                    "contentType": "text"
                    }

        endpoint = f"{BASE_URL}/{list_id}/tasks"
        request_body = {
            "title": task_name,
            "body":body,
            "importance": importance,
            "dueDateTime": datetime_to_api_timestamp(dueDateTime),
            "reminderDateTime": datetime_to_api_timestamp(reminder_datetime),
        }
        session = get_oauth_session()
        response = session.post(endpoint, json=request_body)
        return True if response.ok else response.raise_for_status()

In [15]:
todo = MicrosoftToDo()

In [16]:
todo_tasks = todo.get_tasks(list_name='Tasks')
todo_tasks[0:2]
names = [item['title'] for item in todo_tasks]
print (names)
              

['Mang thêm một hộp kẹp bướm vào ngày mai về nhà', 'lấy tài liệu, chọn nhóm giải thuật nâng cao', 'send meeting file to laptop tomorrow', 'implement code face recognition python', 'reimplement building web service', 'make docker for OCR', 'Check data grip about OPS. What else?', 'Reimplement scaffold prj', 'Lên plan đi đà lạt', 'stock', 'Cloud computing Foundation']


In [17]:
# todo_tasks

In [18]:
todo_lists = todo.get_lists()
todo_lists[0]

Lists:  ['Tasks', '2022', '2023', '2024', 'Giải quyết với Trân', 'LLM RAG and Notion', 'Mua sách', 'Shopping', 'Telegram bot', 'Thạc sĩ', 'Vietnamese stock forecast', 'Flagged Emails']


{'@odata.etag': 'W/"wyYSbsYOaUCZPsQATrx1BgAE6UvZcA=="',
 'displayName': 'Tasks',
 'isOwner': True,
 'isShared': False,
 'wellknownListName': 'defaultList',
 'id': 'AQMkADAwATMwMAItMTg4AGMtZmRiNC0wMAItMDAKAC4AAAORlXbjKGFLSJiBLf4_KVkjAQDDJhJuxg5pQJk_xABOvHUGAAACARIAAAA='}

In [19]:
todo_lists[1]

{'@odata.etag': 'W/"wyYSbsYOaUCZPsQATrx1BgAE5r7y+w=="',
 'displayName': '2022',
 'isOwner': True,
 'isShared': False,
 'wellknownListName': 'none',
 'id': 'AQMkADAwATMwMAItMTg4AGMtZmRiNC0wMAItMDAKAC4AAAORlXbjKGFLSJiBLf4_KVkjAQDDJhJuxg5pQJk_xABOvHUGAAMPg-liAAAA'}

In [20]:
# create_task(task_name='hello', list_name='Tasks', reminder_datetime=datetime.now())

In [21]:
todo_tasks = todo.get_short_form_tasks(list_name='Tasks', )
todo_tasks[0:2]
# names = [item['title'] for item in todo_tasks]
# print (names)
              

[{'importance': 'normal',
  'isReminderOn': False,
  'title': 'Mang thêm một hộp kẹp bướm vào ngày mai về nhà',
  'createdDateTime': '2024-01-17T16:16:02.7843726Z',
  'hasAttachments': False,
  'checklistItems': None,
  'body': {'content': '\r\n', 'contentType': 'text'}},
 {'importance': 'normal',
  'isReminderOn': False,
  'title': 'lấy tài liệu, chọn nhóm giải thuật nâng cao',
  'createdDateTime': '2024-01-17T11:49:14.4019742Z',
  'hasAttachments': False,
  'checklistItems': None,
  'body': {'content': '\r\n', 'contentType': 'text'}}]

In [28]:
task

[{'title': 'Take laptop in Hoa house after lunch',
  'important': True,
  'dueDateTime': None},
 {'title': 'Call friends', 'important': False, 'dueDateTime': None},
 {'title': 'Call parents at 10 o clock tomorrow',
  'important': True,
  'dueDateTime': '2024-01-19:10:00:00'},
 {'title': 'Go to the store at 4PM',
  'important': False,
  'dueDateTime': '2024-01-18:16:00:00'}]

In [24]:
tmp = task[2]
tmp

{'title': 'Call parents at 10 o clock tomorrow',
 'important': True,
 'dueDateTime': '2024-01-19:10:00:00'}

In [27]:
from datetime import datetime
# formatted_date = "2024-10-25"
# datetime_object = datetime.strptime(formatted_date, "%Y-%m-%d")
for tmp in task:
    todo.create_task(task_name=tmp["title"], list_name='Tasks', importance=tmp["important"], dueDateTime=tmp["dueDateTime"])

None
None
{'dateTime': '2024-01-19T03:00:00', 'timeZone': 'UTC'}
{'dateTime': '2024-01-18T09:00:00', 'timeZone': 'UTC'}
